# Processamento de Imagens Digitais com Esqueletização e Poda 

Este notebook implementa algoritmos de processamento morfológico aplicados a uma imagem de impressão digital. Serão aplicadas as etapas de **binarização**, **esqueletização** e **poda** para remover ramos espúrios tentando não comprometer a estrutura principal da digital.

Feito por: Derick Teles Chagas


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, img_as_ubyte

## Funções Morfológicas 

A seguir, definimos funções para realizar operações morfológicas básicas em imagens binárias, como erosão, dilatação e abertura.

In [1]:
def binary_erosion(image, se):
    """
    Realiza a erosão binária da imagem utilizando o elemento estruturante 'se'.
    image: imagem binária (0 e 1)
    se: elemento estruturante (array com 0 e 1)
    """
    se_h, se_w = se.shape
    pad_h = se_h // 2
    pad_w = se_w // 2
    # Padding com zeros (fundo)
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    eroded = np.zeros_like(image)
    
    # Itera sobre todos os pixels
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Seleciona a região da imagem correspondente ao elemento estruturante
            region = padded[i:i+se_h, j:j+se_w]
            # Se todos os pixels da região onde o elemento estruturante tem valor 1 forem 1, o pixel permanece
            if np.all(region[se == 1] == 1):
                eroded[i, j] = 1
    return eroded


def binary_dilation(image, se):
    """
    Realiza a dilatação binária da imagem utilizando o elemento estruturante 'se'.
    """
    se_h, se_w = se.shape
    pad_h = se_h // 2
    pad_w = se_w // 2
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    dilated = np.zeros_like(image)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded[i:i+se_h, j:j+se_w]
            # Se pelo menos um pixel na região onde se == 1 for 1, o pixel é dilatado
            if np.any(region[se == 1] == 1):
                dilated[i, j] = 1
    return dilated


def binary_opening(image, se):
    """
    Realiza a abertura binária, que é a erosão seguida de dilatação.
    """
    eroded = binary_erosion(image, se)
    opened = binary_dilation(eroded, se)
    return opened


## Esqueletização Morfológica

Nesta etapa, calculo o esqueleto da imagem binária. A ideia é erodir a imagem iterativamente e acumular as camadas removidas (S<sub>k</sub>) até que a imagem se torne vazia. 

O esqueleto é dado por:

S<sub>k</sub> = (A ⊖ kB) - ((A ⊖ kB) ∘ B)

Onde ⊖ representa a erosão e ∘ a abertura.

In [2]:
def morphological_skeleton(image, se):
    """
    Calcula o esqueleto morfológico da imagem binária 'image' utilizando o elemento estruturante 'se'.
    """
    img_current = image.copy()
    skeleton = np.zeros_like(image)
    iteration = 0
    
    while np.any(img_current):  # enquanto houver pixels de foreground
        eroded = binary_erosion(img_current, se)
        opened = binary_dilation(eroded, se)
        # S_k: pixels removidos na iteração atual
        S_k = np.logical_and(img_current, np.logical_not(opened)).astype(np.uint8)
        skeleton = np.logical_or(skeleton, S_k).astype(np.uint8)
        img_current = eroded.copy()
        iteration += 1
        # Limite de iterações para evitar laços infinitos
        if iteration > 1000:
            break
    return skeleton


## Funções Auxiliares para Poda Refinada

A poda (pruning) é utilizada para remover ramos espúrios do esqueleto. Nesta implementação, 
a ideia é seguir a ramificação a partir de um endpoint e remover somente aqueles ramos que 
são curtos demais, definidos pelo parâmetro `branch_length_threshold`. Assim, preservamos a estrutura principal da digital.

In [ ]:
def find_endpoints(skel):
    """
    Identifica os pontos finais (endpoints) na imagem esquelética.
    Um pixel é endpoint se ele for 1 e tiver apenas 1 vizinho na vizinhança 8-conectada.
    """
    endpoints = np.zeros_like(skel)
    for i in range(1, skel.shape[0]-1):
        for j in range(1, skel.shape[1]-1):
            if skel[i, j] == 1:
                neighborhood = skel[i-1:i+2, j-1:j+2]
                if (np.sum(neighborhood) - 1) == 1:
                    endpoints[i, j] = 1
    return endpoints


def follow_branch(skel, start, max_length):
    """
    A partir de um endpoint 'start', segue a ramificação enquanto:
      - houver exatamente um vizinho (exceto o pixel anterior)
      - o comprimento não ultrapassar 'max_length'
      - não encontrar uma junção (mais de 1 vizinho)
    Retorna a lista de coordenadas que compõem o ramo.
    """
    branch = [start]
    current = start
    prev = None
    while True:
        i, j = current
        neighbors = []
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                ni, nj = i + di, j + dj
                # Verifica os limites da imagem
                if ni < 0 or ni >= skel.shape[0] or nj < 0 or nj >= skel.shape[1]:
                    continue
                if (ni, nj) == prev:
                    continue
                if skel[ni, nj] == 1:
                    neighbors.append((ni, nj))
        if len(neighbors) == 0:
            break
        if len(neighbors) > 1:
            # Encontrou uma junção; encerra o rastreamento
            break
        next_pixel = neighbors[0]
        branch.append(next_pixel)
        if len(branch) >= max_length:
            break
        prev = current
        current = next_pixel
    return branch


def refined_prune_skeleton(skel, branch_length_threshold=5):
    """
    Remove ramos espúrios da imagem esquelética.
    """
    skel_pruned = skel.copy()
    endpoints = find_endpoints(skel_pruned)
    endpoint_coords = np.argwhere(endpoints == 1)
    for coord in endpoint_coords:
        i, j = coord
        branch = follow_branch(skel_pruned, (i, j), branch_length_threshold + 1)
        if len(branch) <= branch_length_threshold:
            for (p, q) in branch:
                skel_pruned[p, q] = 0
    return skel_pruned


## Processamento da Imagem

Nesta seção, carregamos a imagem da impressão digital, convertemos para escala de cinza, 
normalizamos e binarizamos utilizando um limiar adequado.

In [ ]:
img = io.imread('digital.png')

# Se a imagem estiver em RGB, converte para escala de cinza
if img.ndim == 3:
    img_gray = color.rgb2gray(img)
else:
    img_gray = img

# Converte para 8-bit e normaliza para [0, 1]
img_gray = img_as_ubyte(img_gray)
img_norm = img_gray / 255.0

# Binarização: ajuste o threshold conforme necessário (ex: 0.5)
threshold = 0.5
img_bin = (img_norm > threshold).astype(np.uint8)

# Exibe a imagem original e a imagem binarizada
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img_gray, cmap='gray')
ax[0].set_title('Imagem Original (Escala de Cinza)')
ax[0].axis('off')

ax[1].imshow(img_bin, cmap='gray')
ax[1].set_title('Imagem Binarizada')
ax[1].axis('off')

plt.tight_layout()
plt.show()

## Aplicação dos Algoritmos: Esqueletização e Poda Refinada

A seguir, aplicamos os algoritmos de esqueletização e poda refinada à imagem binarizada.

In [ ]:
# Escolha do elemento estruturante (3x3 com todos os pixels iguais a 1)
se = np.ones((3, 3), dtype=np.uint8)

# Esqueletização
skeleton = morphological_skeleton(img_bin, se)

# Poda refinada: ajustando o parâmetro branch_length_threshold conforme necessário
pruned_skeleton = refined_prune_skeleton(skeleton, branch_length_threshold=5)

# Exibição das 4 imagens lado a lado
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Imagem Original')
axes[0].axis('off')

axes[1].imshow(img_bin, cmap='gray')
axes[1].set_title('Imagem Binarizada')
axes[1].axis('off')

axes[2].imshow(skeleton, cmap='gray')
axes[2].set_title('Esqueletização')
axes[2].axis('off')

axes[3].imshow(pruned_skeleton, cmap='gray')
axes[3].set_title('Esqueleto Após Poda Refinada')
axes[3].axis('off')

plt.tight_layout()
plt.show()

## Conclusão

Neste notebook, demonstramos como implementar operações morfológicas para realizar a esqueletização e uma poda refinada que remove ramos espúrios tentando não comprometer a estrutura principal da impressão digital.

